In [4]:
import tweepy
import pandas as pd
from scipy.misc import imread

import matplotlib as mpl
import csv
import matplotlib.pyplot as plt

import nltk 
from nltk.corpus import stopwords
import string
import pandas as pd
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
import re

### Twitter Authentication:
https://beginanalyticsblog.wordpress.com/2018/02/07/twitter-data-analysis-using-python/

In [1]:

# Authenticating my account to access twitter’s API.
consumer_key = "LtTYnj1rtVRNpcxVKnB0QhqmZ"
consumer_secret = "DOUo6wXpG5RnivUYiR2iTP04qg1wNfcgwsKQf6GE5hru01phyR"
access_token = "367018485-YIzt7FonHnU292KZWDHBu2QdFpZOQHpDGo6e1kAc"
access_token_secret = "maJFd0ge7SY3c3tljWxCvzw7SZiOybpaOo1I6aLGocLJj"


In [2]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

In [4]:
# Search term that you want to find
query = "iphone8"

# Language code to select a language
language = "en"

#Extracting tweets from twitter and appending to result object.
results = []
for tweet in tweepy.Cursor (api.search, q = 'iphone8', lang = "en").items(1500): 
    results.append(tweet)
    

<class 'list'>
1493


### Data Preparation and Cleaning:
Store the relevant tweets data in a data frame and remove tweets that have duplicate text. You may find tweets that have the exact same text but have been re-tweeted by different users.

In [5]:
#Extracting elements from the tweet and populating data sets
def tweets_df(results):
    
    id_list = [tweet.id for tweet  in results]
    data_set = pd.DataFrame(id_list, columns = ["id"])
    
    #Creating datafarmes for tweet fields.
    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]
    data_set["user_verified"] = [tweet.author.verified for tweet in results]
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in results]
    data_set["user_listed_count"] = [tweet.author.listed_count for tweet in results]
    data_set["user_favourites_count"] = [tweet.author.favourites_count for tweet in results]
    data_set[""] = [tweet.author.statuses_count for tweet in results]
    data_set["user_geo_enabled"] = [tweet.author.geo_enabled for tweet in results]
    data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in results]
    
    return data_set


#Getting the extracted tweets and removing fields from them.
data_set = tweets_df(results)


# Remove tweets with duplicate
text = data_set["text"]

for i in range(0,len(text)):
    txt = ' '.join(word for word in text[i] .split() if not word.startswith('https:'))
    data_set.set_value(i,'text2', txt)
    
data_set.drop_duplicates('text2', inplace=True)
data_set.reset_index(drop = True, inplace=True)
data_set.drop('text', axis = 1, inplace = True)
data_set.rename(columns={'text2': 'text'}, inplace=True)

In [8]:
data_set.to_csv('output294.csv')

We take this data and mannually label them as bot or no bot and create a file 'outputbot.csv'

We use Beautiful soup for data preparation.

In [37]:
tok = WordPunctTokenizer()

data1= pd.read_csv('outputbot.csv')

#removing @ mentions
pat1 = r'@[A-Za-z0-9]+'
#removing links
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    # Remove HTML encoding that has been converted to text
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    #decoding anny UTF-8 encoding and replacing the characters by ?
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    
    #cleaning the hash tags and numbers
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    #covering text to lower case
    lower_case = letters_only.lower()
    # Removing unnecessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()



#removing stop words from tweets
def msg_transform(message):
    #Takes in a string of text, then performs the following:
    #1. Remove all punctuation
    #2. Remove all stopwords
    #3. Returns a list of the cleaned text
    # Check characters to see if they are in punctuation
    rmv_punc = [char for char in message if char not in string.punctuation]

    # Join the characters again to form the string.
    rmv_punc = ''.join(rmv_punc)
    
    # Now just remove any stopwords
    return [word for word in rmv_punc.split() if word.lower() not in stopwords.words('english')]


#we clean the tweets and then remove stop words from them and the output the csv file to be used by our Bag of words algorithm.
testing = data1['text']
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))


test_result1 = []
for t in test_result:
    test_result1.append(msg_transform(t))


test_result1
my_df = pd.DataFrame(test_result1)
my_df.to_csv('bow.csv')





In [38]:
#We store the result of cleaned tweets to file. 
my_df = pd.DataFrame(test_result)
my_df.to_csv('clean.csv')

In [37]:
print(my_df)

                                                     0
0    all your problems to watch ipl matches can be ...
1            is iphone or samsung s worth to buy today
2                                       rt iphone cool
3                                               iphone
4                                        nicchi iphone
5    win one of these iphonex iphone just retweet f...
6    rt we re excited to announce our newest giveaw...
7    paige i m going to get iphone plus red it s pa...
8             rt y all see what a iphone can do prom k
9    we d love to work with you to get the iphone f...
10               y all see what a iphone can do prom k
11   rt iphone x space grey apple iphonex ios iphon...
12   iphone x space grey apple iphonex ios iphoneph...
13   im playing fortnite on my iphone the future is...
14   win one of these iphonex iphone just retweet f...
15   no filter one shot of this gorgeous east longm...
16   yesterday i checked out iphone product red wit...
17        

References:
https://towardsdatascience.com/another-twitter-sentiment-analysis-bb5b01ebad90
https://www.analyticsvidhya.com/blog/2014/11/text-data-cleaning-steps-python/